# RESNET 50

## References 
1. https://www.kaggle.com/parthdhameliya77/pytorch-resnext50-32x4d-image-tfidf-inference
2. https://www.kaggle.com/karthur10/resnet-sbert-unsupervised-baseline


## Importing required libraries like torch, cuml, cv2, cudf etc

In [ ]:
import numpy as np
import cupy, cudf
import gc
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import random
import torch
import torchvision
from torchvision import  models, transforms
from transformers import BertTokenizer, BertModel
from cuml.feature_extraction.text import TfidfVectorizer
from cuml.neighbors import NearestNeighbors
import torch.nn as nn
import torch.nn.functional as F
import os
import glob
from PIL import Image
import seaborn as sns
import cv2, matplotlib.pyplot as plt
import matplotlib.image as mpimg
from textwrap import wrap

## Importing Test and Train Image Data

In [ ]:
device = 'cuda'if torch.cuda.is_available() else 'cpu'
device

In [ ]:
PATH = '../input/shopee-product-matching/'
PATH_TO_IMG = '../input/shopee-product-matching/train_images/'
PATH_TO_TEST = '../input/shopee-product-matching/test_images/'
os.listdir(PATH)

In [ ]:
COMPUTE_CV = True
if len(pd.read_csv(PATH + 'test.csv')) > 3: COMPUTE_CV = False

In [ ]:
if COMPUTE_CV:
    dataset = pd.read_csv(PATH + 'train.csv')
    tmp = dataset.groupby('label_group').posting_id.agg('unique').to_dict()
    dataset['target'] = dataset.label_group.map(tmp)
else:    
    dataset = pd.read_csv(PATH + 'test.csv')

In [ ]:
dataset.head()

## Display Random Images using CV2

In [ ]:
def show_random_img():
    # choose randomly two instances per each class
    labels_to_show = np.random.choice(dataset.label_group.unique(), 
                                      replace=False, size=24)
    img_to_show = []
    for label in labels_to_show:
        rows = dataset[dataset.label_group==label].copy()
        pair = np.random.choice([i for i in range(len(rows))], 
                                    replace=False, size=2)
        img_pair = rows.iloc[pair][['image', 'title']].values
        
        img_to_show += list(img_pair)
    
    fig, axes = plt.subplots(figsize = (18, 12), nrows=4,ncols=6)
    for imp, ax in zip(img_to_show, axes.ravel()):
        img = cv2.imread(PATH_TO_IMG + imp[0])
        title = '\n'.join(wrap(imp[1], 20))
        ax.set_title(title)
        ax.imshow(img)
        ax.axis('off')

    fig.tight_layout()

In [ ]:
if COMPUTE_CV:
    show_random_img()

# ResNet50 block with tranform using Pretrained model,

In [ ]:
class ResNetEmbedder(nn.Module):
    
    def __init__(self, device='cpu'):
        super(ResNetEmbedder, self).__init__()
        self.model = models.resnet50(pretrained=False)
        self.device = device
        path = '../input/pretrained-model-weights-pytorch/resnet50-19c8e357.pth'
        self.model.load_state_dict(torch.load(path))
#         to freeze weights
        for param in self.model.parameters():
                param.requires_grad = False
        self.model.to(device)
        
    
    def transform(self, img):
        image_transform = torchvision.transforms.Compose(
            [
                torchvision.transforms.Resize(256),
                transforms.CenterCrop(224),
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize(
                    mean=(0.485, 0.456, 0.406), 
                    std=(0.229, 0.224, 0.225)
                ),
            ]
        )
        return image_transform(img)
    
    def forward(self, img):
        img_tr = self.transform(img).unsqueeze(0)
        img_tr = img_tr.to(self.device)
        features = self.model(img_tr).squeeze()
        return features

In [ ]:
model_img = ResNetEmbedder(device)

In [ ]:
def vectorize_img(img_path):
    img = Image.open(img_path).convert('RGB')
    model_img.eval()
    with torch.no_grad():
        output = model_img(img).cpu().numpy()
    return output

In [ ]:
%%time
if COMPUTE_CV:
    dataset['resnet_v'] = dataset['image'].progress_apply(lambda x: vectorize_img(PATH_TO_IMG + x))
else:
    dataset['resnet_v'] = dataset['image'].progress_apply(lambda x: vectorize_img(PATH_TO_TEST + x))

In [ ]:
del model_img

In [ ]:
print(dataset)

## Normalizing the vectors

In [ ]:
vectors = np.stack(dataset.resnet_v)
vectors = torch.Tensor(vectors).to(device)
vectors = F.normalize(vectors)

## Finding similar titles in chunks of 1024

In [ ]:
preds = []
CHUNK = 1024

print('Finding similar titles...')
CTS = len(dataset)//CHUNK
if len(dataset)%CHUNK!=0: CTS += 1
for j in range( CTS ):
    
    a = j*CHUNK
    b = (j+1)*CHUNK
    b = min(b,len(dataset))
    print('chunk',a,'to',b)
    
    # COSINE SIMILARITY DISTANCE
    cts = torch.matmul( vectors, vectors[a:b].T).T
    cts = cts.cpu().numpy()
    
    for k in range(b-a):
        IDX = np.where(cts[k,]>0.9)[0]
        o = dataset.iloc[IDX].posting_id.values
        preds.append(o)

del vectors, cts, IDX, o
_ = gc.collect()

## Predictions and computation of F1 score

In [ ]:
dataset['preds_resnet'] = preds
dataset.head()

In [ ]:
def getMetric(col):
    def f1score(row):
        n = len( np.intersect1d(row.target,row[col]) )
        return 2*n / (len(row.target)+len(row[col]))
    return f1score

In [ ]:
if COMPUTE_CV:
    dataset['f1_resnet'] = dataset.apply(getMetric('preds_resnet'), axis=1)
    print('F1 Score =', dataset.f1_resnet.mean())